# Basic -> "excavator_data.xlsx"


## Detail한 정보는 없음. (Just Model and Info)

In [4]:
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import openpyxl
from selenium.webdriver.chrome.options import Options
from urllib.parse import urlparse
from selenium.common.exceptions import NoSuchElementException

# Function to extract domain from URL
def extract_domain(url):
    parsed_url = urlparse(url)
    return parsed_url.netloc

# Function to set up the WebDriver with anti-detection measures
def setup_driver():
    options = Options()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("start-maximized")
    # Randomized User-Agent
    options.add_argument(f"user-agent=Mozilla/5.0 (Windows NT {random.randint(6, 10)}.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/{random.randint(90, 110)}.0.4472.{random.randint(100, 200)} Safari/537.36")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver

# Function to add random delay
def random_delay(min_seconds=1, max_seconds=3):
    time.sleep(random.uniform(min_seconds, max_seconds))

# Main scraping logic
def scrape_excavator_data():
    url = "https://asia.develon-ce.com/en/equipment/excavators/"
    driver = setup_driver()
    driver.get(url)
    
    # Create Excel workbook
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.title = "Excavator Data"
    sheet.append(["Model", "Info", "Details"])
    
    while True:
        try:
            # Extract models and info
            model_elements = driver.find_elements(By.CSS_SELECTOR, ".product_name.en_plain_b")
            info_elements = driver.find_elements(By.CSS_SELECTOR, ".product_info.en_plain_l")
            
            if len(model_elements) != len(info_elements):
                print("Mismatch in data lengths. Exiting.")
                break
            
            for model, info in zip(model_elements, info_elements):
                # Simulate human-like delay
                random_delay()
                sheet.append([model.text, info.text, "Details Placeholder"])  # Placeholder for details
                
            # Click 'Load More' button
            try:
                load_more_button = driver.find_element(By.CLASS_NAME, "btn_more")
                ActionChains(driver).move_to_element(load_more_button).perform()
                random_delay()
                load_more_button.click()
                random_delay(2, 4)  # Longer delay for content load
            except NoSuchElementException:
                print("No 'Load More' button. Exiting loop.")
                break
        except Exception as e:
            print(f"Error: {e}")
            break
    
    # Save data to Excel
    file_name = "excavator_data.xlsx"
    workbook.save(file_name)
    print(f"Data saved to {file_name}")
    
    # Close browser
    driver.quit()

# Run the script
if __name__ == "__main__":
    scrape_excavator_data()


No 'Load More' button. Exiting loop.
Data saved to excavator_data.xlsx


# Type1 (첫번째에는 잘 작동됐는데 captcha문제로 지금은 안됨.)

In [12]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urlparse
import pandas as pd
import openpyxl

# Function to extract domain from URL
def extract_domain(url):
    parsed_url = urlparse(url)
    return parsed_url.netloc

# Function to search Google
def search_google(query):
    search_url = f"https://www.google.com/search?q={query}"
    
    # Set up the Chrome WebDriver
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless")  # Run in headless mode (without opening a browser window)
    # options.add_argument("--disable-gpu")
    # options.add_argument("--no-sandbox")
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(search_url)
    
    # Wait for results to load
    time.sleep(3)  # Wait for the page to load, adjust if needed
    
    return driver

# Function to get the first search result
def get_first_search_result(url, model):
    query = extract_domain(url) + "+" + model
    driver = search_google(query)
    
    # Find the first search result using the href attribute
    try:
        first_result = driver.find_element(By.CSS_SELECTOR, 'a[jsname="UWckNb"]')
        link = first_result.get_attribute('href')  # Extract the href attribute (the URL)
        return link
    except Exception as e:
        return f"Error: {e}"
    finally:
        driver.quit()  # Close the browser when done

# Initialize the WebDriver for the first website
options = Options()
b = webdriver.Chrome(options=options)

# Open the target URL
url = "https://asia.develon-ce.com/en/equipment/excavators/"
b.get(url)

# Create a new Excel workbook and sheet
workbook = openpyxl.Workbook()
table = workbook.active
element = b.find_element(By.CLASS_NAME, "en_plain_b")  # Get the WebElement
table.title = element.text + "data"  # Concatenate its text content with "data"

# Add headers to the Excel sheet
table.append(["Model", "Info", "Details"])

# Extract data and insert into the Excel sheet
while True:
    try:
        # Find elements for model names and info
        model_elements = b.find_elements(By.CSS_SELECTOR, ".product_name.en_plain_b")
        info_elements = b.find_elements(By.CSS_SELECTOR, ".product_info.en_plain_l")

        # Check if both lists have the same length
        if len(model_elements) != len(info_elements):
            print("Mismatch in number of model and info elements. Exiting loop.")
            break

        # Add data to the Excel sheet
        for model, info in zip(model_elements, info_elements):
            # Get the first search result for the model
            first_result = get_first_search_result(url, model.text)

            # Extract details from the first result
            try:
                tables = pd.read_html(first_result)
                combined_table = pd.concat([tables[0], tables[1]], axis=1)  # Combine tables if needed
                combined_table_transposed = combined_table.T  # Transpose the combined table
                details = combined_table_transposed
            except Exception as e:
                details = "Details not found"

            # Append model, info, and details to the sheet
            table.append([model.text, info.text, details])

        # Check if a "Load More" button exists
        try:
            load_more_button = b.find_element(By.CLASS_NAME, "btn_more")
            ActionChains(b).move_to_element(load_more_button).perform()
            time.sleep(0.5)
            load_more_button.click()
            time.sleep(2)  # Wait for content to load
        except NoSuchElementException:
            print("No 'Load More' button found. Exiting loop.")
            break  # Exit loop if no "Load More" button is found
    except Exception as e:
        print(f"Error: {e}")
        break

# Save the workbook to an Excel file
file_name = "excavator_data_with_details(0).xlsx"
workbook.save(file_name)
print(f"Data successfully saved to {file_name}")

# Close the WebDriver
b.quit()

Error: Cannot convert                               0                             1   \
Classification  Operating Weight  Bucket Capacity (SAE HEAPED)   
DX89R-7                 9,518 kg                       0.28 m³   

                                           2             3             4   \
Classification  Engine Power (SAE j1349, net)  Engine Model  Travel Speed   
DX89R-7                   48.5/2,100 (kW/rpm)   Develon D24  2.7/4.7 km/h   

                                5                   6                    7   \
Classification  Max. Digging Reach  Max. Digging Depth  Max. Digging Height   
DX89R-7                   7,015 mm            4,110 mm             7,030 mm   

                            8              9               10  
Classification  Overall Length  Overall Width  Overall Height  
DX89R-7               6,430 mm       2,250 mm        2,657 mm   to Excel
Data successfully saved to excavator_data_with_details(0).xlsx


# Type2 -> "excator_data_with_link.xlsx" (지금은 링크만 존재) (40~60min)

## Type1의 captcha 해결 버전

 Kept failing so I checked whether if it uses correct url and apparently google blocks after certain usage

In [6]:
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urlparse
from selenium.common.exceptions import NoSuchElementException
import openpyxl
from selenium.webdriver.chrome.options import Options

# Function to extract domain from URL
def extract_domain(url):
    parsed_url = urlparse(url)
    return parsed_url.netloc

# Function to set up the WebDriver with anti-detection measures
def setup_driver(headless=False):
    options = Options()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("start-maximized")
    # Randomized User-Agent
    options.add_argument(f"user-agent=Mozilla/5.0 (Windows NT {random.randint(6, 10)}.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/{random.randint(90, 110)}.0.4472.{random.randint(100, 200)} Safari/537.36")
    if headless:
        options.add_argument("--headless")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver

# Function to add random delay
def random_delay(min_seconds=1, max_seconds=3):
    time.sleep(random.uniform(min_seconds, max_seconds))

# Function to search Google and get the first result
def search_google(query):
    search_url = f"https://www.google.com/search?q={query}"
    driver = setup_driver(headless=True)
    driver.get(search_url)
    random_delay(2, 4)
    try:
        # Find the first search result
        first_result = driver.find_element(By.CSS_SELECTOR, 'a[jsname="UWckNb"]')
        link = first_result.get_attribute('href')  # Extract the href attribute (the URL)
        return link
    except Exception as e:
        return f"Error: {e}"
    finally:
        driver.quit()

# Main scraping logic
def scrape_excavator_data():
    url = "https://asia.develon-ce.com/en/equipment/excavators/"
    driver = setup_driver()
    driver.get(url)
    
    # Create Excel workbook
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.title = "Excavator Data"
    sheet.append(["Model", "Info", "Details"])

    while True:
        try:
            # Extract models and info
            model_elements = driver.find_elements(By.CSS_SELECTOR, ".product_name.en_plain_b")
            info_elements = driver.find_elements(By.CSS_SELECTOR, ".product_info.en_plain_l")
            
            if len(model_elements) != len(info_elements):
                print("Mismatch in data lengths. Exiting.")
                break
            
            for model, info in zip(model_elements, info_elements):
                random_delay()
                model_text = model.text
                info_text = info.text
                # Search Google for additional details
                query = extract_domain(url) + "+" + model_text
                details = search_google(query)
                sheet.append([model_text, info_text, details])
            
            # Click 'Load More' button
            try:
                load_more_button = driver.find_element(By.CLASS_NAME, "btn_more")
                ActionChains(driver).move_to_element(load_more_button).perform()
                random_delay()
                load_more_button.click()
                random_delay(2, 4)  # Longer delay for content load
            except NoSuchElementException:
                print("No 'Load More' button. Exiting loop.")
                break
        except Exception as e:
            print(f"Error: {e}")
            break

    # Save data to Excel
    file_name = "excavator_data_with_details.xlsx"
    workbook.save(file_name)
    print(f"Data saved to {file_name}")
    
    # Close browser
    driver.quit()

# Run the script
if __name__ == "__main__":
    scrape_excavator_data()


No 'Load More' button. Exiting loop.
Data saved to excavator_data_with_details.xlsx


# Final Version

In [29]:
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urlparse
import openpyxl
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm  # Import tqdm for progress bar

# Function to extract domain from URL
def extract_domain(url):
    parsed_url = urlparse(url)
    return parsed_url.netloc

# Function to search Google
def search_google(query):
    search_url = f"https://www.google.com/search?q={query}"
    
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("start-maximized")
    options.add_argument(f"user-agent=Mozilla/5.0 (Windows NT {random.randint(6, 10)}.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/{random.randint(90, 110)}.0.4472.{random.randint(100, 200)} Safari/537.36")
        
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(search_url)
    return driver

# Add a delay to mimic human behavior
def random_delay(min_seconds=1, max_seconds=3):
    time.sleep(random.uniform(min_seconds, max_seconds))

def get_first_search_result(url, model):
    query = extract_domain(url) + "+" + model
    driver = search_google(query)
    
    try:
        first_result = driver.find_element(By.CSS_SELECTOR, 'a[jsname="UWckNb"]')
        link = first_result.get_attribute('href')
        return link
    except Exception as e:
        return f"Error: {e}"
    finally:
        driver.quit()

# Main function to scrape excavator data
def scrape_excavator_data():
    url = "https://asia.develon-ce.com/en/equipment/excavators/"
    options = webdriver.ChromeOptions()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)

    # Create Excel workbook
    workbook = openpyxl.Workbook()
    table = workbook.active
    table.title = "Excavator Data"
    table.append(["Model"] + [f"Detail {i}" for i in range(1, 20)])  # Header row (adjust number of details as needed)

    while True:
        try:
            model_elements = driver.find_elements(By.CSS_SELECTOR, ".product_name.en_plain_b")

            for model in tqdm(model_elements, total=len(model_elements), desc="Scraping Models"):
                random_delay()
                first_result = get_first_search_result(url, model.text)

                try:
                    tables = pd.read_html(first_result)
                    detail_info = tables[1]  # Assuming details are in the second table
                    flattened_details = detail_info.values.flatten().tolist()  # Flatten DataFrame into a single list
                except Exception as e:
                    flattened_details = ["Details not found"]

                # Append model and flattened details in one row
                table.append([model.text] + flattened_details)
            
            try:
                load_more_button = driver.find_element(By.CLASS_NAME, "btn_more")
                ActionChains(driver).move_to_element(load_more_button).perform()
                random_delay()
                load_more_button.click()
                random_delay(2, 4)  # Longer delay for content load
            except NoSuchElementException:
                print("No 'Load More' button. Exiting loop.")
                break

        except Exception as e:
            print(f"Error: {e}")
            break

    file_name = "excavator_data_with_details.xlsx"
    workbook.save(file_name)
    print(f"Data successfully saved to {file_name}")

    driver.quit()

if __name__ == "__main__":
    scrape_excavator_data()


Scraping Models:   0%|          | 0/29 [00:19<?, ?it/s]


KeyboardInterrupt: 

# 보험

In [17]:
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urlparse
import openpyxl
import pandas as pd
from selenium.common.exceptions import NoSuchElementException

# Function to extract domain from URL
def extract_domain(url):
    parsed_url = urlparse(url)
    return parsed_url.netloc

# Function to set up the WebDriver with anti-detection measures

def search_google(query):
    search_url = f"https://www.google.com/search?q={query}"
    
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("start-maximized")
    # Randomized User-Agent
    options.add_argument(f"user-agent=Mozilla/5.0 (Windows NT {random.randint(6, 10)}.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/{random.randint(90, 110)}.0.4472.{random.randint(100, 200)} Safari/537.36")
        
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    driver.get(search_url)

    return driver
    

def random_delay(min_seconds=1, max_seconds=3):
    time.sleep(random.uniform(min_seconds, max_seconds))

def get_first_search_result(url, model):
    query = extract_domain(url) + "+" + model
    driver = search_google(query)
    
    # Find the first search result using the href attribute
    try:
        first_result = driver.find_element(By.CSS_SELECTOR, 'a[jsname="UWckNb"]')
        link = first_result.get_attribute('href')  # Extract the href attribute (the URL)
        return link
    
    except Exception as e:
        return f"Error: {e}"
    finally:
        driver.quit()  # Close the browser when done

# Function to scrape excavator data
def scrape_excavator_data():
    url = "https://asia.develon-ce.com/en/equipment/excavators/"
    driver = setup_driver()
    driver.get(url)

    # Create Excel workbook
    workbook = openpyxl.Workbook()
    table = workbook.active
    table.title = "Excavator Data"
    table.append(["Model", "Info", "Detail"])

    while True:
        try:
            # Extract models and info
            model_elements = driver.find_elements(By.CSS_SELECTOR, ".product_name.en_plain_b")
            info_elements = driver.find_elements(By.CSS_SELECTOR, ".product_info.en_plain_l")

            if len(model_elements) != len(info_elements):
                print("Mismatch in data lengths. Exiting.")
                break

            for model, info in zip(model_elements, info_elements):
                random_delay()
                first_result = get_first_search_result(url,model.text)

                try:
                    tables = pd.read_html(first_result)
                    # detail_title_raw = tables[0]
                    # detail_title_transposed = detail_title_raw.T
                    # detail_title = detail_title_transposed

                    detail_info = tables[1]
                    detail_info_transposed = detail_info.T
                    details = detail_info_transposed.to_string(index=False, header=False)
                
                except Exception as e:
                    details = "Details not found"
                
                table.append([model.text, info.text, details])

            try:
                load_more_button = driver.find_element(By.CLASS_NAME, "btn_more")
                ActionChains(driver).move_to_element(load_more_button).perform()
                random_delay()
                load_more_button.click()
                random_delay(2, 4)  # Longer delay for content load
            except NoSuchElementException:
                print("No 'Load More' button. Exiting loop.")
                break
        except Exception as e:
            print(f"Error: {e}")
            break

    file_name = "excavator_data_with_details(2).xlsx"
    workbook.save(file_name)
    print(f"Data successfully saved to {file_name}")

    driver.quit()

if __name__ == "__main__":
    scrape_excavator_data()

NameError: name 'setup_driver' is not defined

# TEST (Bypass Captcha tool used)

In [ ]:
import time
from selenium.webdriver import Remote, ChromeOptions
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chromium.remote_connection import ChromiumRemoteConnection
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import WebDriverException
from urllib.parse import urlparse
import pandas as pd
import openpyxl

AUTH = 'brd-customer-hl_e5bd583b-zone-ai_sraper:n1arzv61qr2a'
SBR_WEBDRIVER = f'https://{AUTH}@brd.superproxy.io:9515'

# Function to extract domain from URL
def extract_domain(url):
    parsed_url = urlparse(url)
    return parsed_url.netloc

# Function to search Google
def search_google(query):
    search_url = f"https://www.google.com/search?q={query}"
    
    # Set up Chrome options
    options = ChromeOptions()
    options.add_argument("--headless")  # Run in headless mode
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument(f"--proxy-server=http://{AUTH.split(':')[0]}")  # Use proxy
    
    driver = None  # Initialize driver variable
    
    try:
        # Create a Remote WebDriver instance
        driver = Remote(command_executor=SBR_WEBDRIVER, options=options)
        driver.get(search_url)
        
        # Wait for the page to load and return the HTML
        time.sleep(3)  # You can use WebDriverWait for better control
        html = driver.page_source
        return html
    
    except WebDriverException as e:
        print(f"Error during WebDriver execution: {e}")
        return None
    
    finally:
        # Ensure the driver is quit properly
        if driver:
            try:
                driver.quit()
            except Exception as e:
                print(f"Error while quitting WebDriver: {e}")

# Function to get the first search result
def get_first_search_result(url, model):
    query = extract_domain(url) + "+" + model
    driver = search_google(query)

    if driver is None:
        return "Error: Could not complete Google search."

    try:
        # Find the first search result using the href attribute
        first_result = driver.find_element(By.CSS_SELECTOR, 'a[jsname="UWckNb"]')
        link = first_result.get_attribute('href')  # Extract the href attribute (the URL)
        return link
    except Exception as e:
        return f"Error: {e}"
    finally:
        driver.quit()  # Ensure the driver is properly closed



# Initialize the WebDriver for the first website
options = Options()
b = webdriver.Chrome(options=options)

# Open the target URL
url = "https://asia.develon-ce.com/en/equipment/excavators/"
b.get(url)



# Create a new Excel workbook and sheet
workbook = openpyxl.Workbook()
table = workbook.active
element = b.find_element(By.CLASS_NAME, "en_plain_b")  # Get the WebElement
table.title = element.text + "data"  # Concatenate its text content with "data"

# Add headers to the Excel sheet
table.append(["Model", "Info", "Details"])

# Extract data and insert into the Excel sheet
while True:
    try:
        # Find elements for model names and info
        model_elements = b.find_elements(By.CSS_SELECTOR, ".product_name.en_plain_b")
        info_elements = b.find_elements(By.CSS_SELECTOR, ".product_info.en_plain_l")

        # Check if both lists have the same length
        if len(model_elements) != len(info_elements):
            print("Mismatch in number of model and info elements. Exiting loop.")
            break

        # Add data to the Excel sheet
        for model, info in zip(model_elements, info_elements):
            # Get the first search result for the model
            first_result = get_first_search_result(url, model.text)

            # Extract details from the first result
            try:
                tables = pd.read_html(first_result)
                combined_table = pd.concat([tables[0], tables[1]], axis=1)  # Combine tables if needed
                combined_table_transposed = combined_table.T  # Transpose the combined table
                details = combined_table_transposed
            except Exception as e:
                details = "Details not found"

            # Append model, info, and details to the sheet
            table.append([model.text, info.text, details])

        # Check if a "Load More" button exists
        try:
            load_more_button = b.find_element(By.CLASS_NAME, "btn_more")
            ActionChains(b).move_to_element(load_more_button).perform()
            time.sleep(0.5)
            load_more_button.click()
            time.sleep(2)  # Wait for content to load
        except NoSuchElementException:
            print("No 'Load More' button found. Exiting loop.")
            break  # Exit loop if no "Load More" button is found
    except Exception as e:
        print(f"Error: {e}")
        break

# Save the workbook to an Excel file
file_name = "excavator_data_with_details.xlsx"
workbook.save(file_name)
print(f"Data successfully saved to {file_name}")

# Close the WebDriver
b.quit()


KeyboardInterrupt: 

# Comment

Try Trial 1.1 & 1.2 & 3.1 & 3.2